# Introducción

Film Junky Union, una nueva comunidad vanguardista para amantes de las películas clásicas, está desarrollando un sistema para filtrar y categorizar reseñas de películas.

## Objetivo

El objetivo es entrenar un modelo para detectar las críticas negativas de forma automática. Utilizarás un conjunto de datos de reseñas de películas de IMDB con leyendas de polaridad para construir un modelo para clasificar las reseñas positivas y negativas.

## Condiciones

Alcanzar un valor F1 de al menos 0.85.

## Tabla de contenido

1. Introducción
2. Preprocesamiento
3. Procesamiento de lenguaje natural
4. Modelos de clasificación
5. Conclusiones

# Preprocesamiento

Comenzaremos cargando los datos del archivo `imdb_reviews.tsv` y analizaremos tanto el contenido como las condiciones en las que se encuentran los datos.

In [28]:
# Importando librerias
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

In [2]:
# Leyendo archivo con extensión "tsv"
df = pd.read_csv('/datasets/imdb_reviews.tsv',sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47331 entries, 0 to 47330
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           47331 non-null  object 
 1   title_type       47331 non-null  object 
 2   primary_title    47331 non-null  object 
 3   original_title   47331 non-null  object 
 4   start_year       47331 non-null  int64  
 5   end_year         47331 non-null  object 
 6   runtime_minutes  47331 non-null  object 
 7   is_adult         47331 non-null  int64  
 8   genres           47331 non-null  object 
 9   average_rating   47329 non-null  float64
 10  votes            47329 non-null  float64
 11  review           47331 non-null  object 
 12  rating           47331 non-null  int64  
 13  sp               47331 non-null  object 
 14  pos              47331 non-null  int64  
 15  ds_part          47331 non-null  object 
 16  idx              47331 non-null  int64  
dtypes: float64(2

In [3]:
# Visualizando contenido de los primeros 10 registros
df.head(10)

,tconst,title_type,primary_title,original_title,start_year,end_year,runtime_minutes,is_adult,genres,average_rating,votes,review,rating,sp,pos,ds_part,idx
0,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,The pakage implies that Warren Beatty and Gold...,1,neg,0,train,8335
1,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,How the hell did they get this made?! Presenti...,1,neg,0,train,8336
2,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,There is no real story the film seems more lik...,3,neg,0,test,2489
3,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,Um .... a serious film about troubled teens in...,7,pos,1,test,9280
4,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,I'm totally agree with GarryJohal from Singapo...,9,pos,1,test,9281
5,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,This is the first movie I've seen from Singapo...,9,pos,1,test,9282
6,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,Yes non-Singaporean's can't see what's the big...,9,pos,1,test,9283
7,tt0035958,movie,'Gung Ho!': The Story of Carlson's Makin Islan...,'Gung Ho!': The Story of Carlson's Makin Islan...,1943,\N,88,0,"Drama,History,War",6.1,1240.0,This true story of Carlson's Raiders is more o...,2,neg,0,train,9903
8,tt0035958,movie,'Gung Ho!': The Story of Carlson's Makin Islan...,'Gung Ho!': The Story of Carlson's Makin Islan...,1943,\N,88,0,"Drama,History,War",6.1,1240.0,Should have been titled 'Balderdash!' Little i...,2,neg,0,train,9905
9,tt0035958,movie,'Gung Ho!': The Story of Carlson's Makin Islan...,'Gung Ho!': The Story of Carlson's Makin Islan...,1943,\N,88,0,"Drama,History,War",6.1,1240.0,The movie 'Gung Ho!': The Story of Carlson's M...,4,neg,0,train,9904


De acuerdo con el objetivo que solicita `Film Junky Union` no tenemos que hacer uso de toda la información presentada en `df`, por lo que las columnas de interes son las siguientes: 

- `review`: la cuál contiene las reseñas de las peliculas.
- `pos`: reseña positiva (1) o negativa (0) la cuál es nuestra columna objetivo.
- `ds_part`: Nos indica que registros serán utilizados para entrenamiento y cuales para testeo.

Una vez teniendo dicha información podemos resaltar hasta este punto que no se tienen registros nulos en las columnas de interes, por lo que procederemos a reducir `df` únicamente con información de relevancia para el cumplimiento del objetivo.

In [4]:
# Ajusta "df" con las columnas de interes
df = df[['review','pos','ds_part']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47331 entries, 0 to 47330
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   47331 non-null  object
 1   pos      47331 non-null  int64 
 2   ds_part  47331 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


Una vez que tenemos la información necesaria, verificaremos lo siguiente:

- Validaremos que no hay duplicidad de registros.
- Verificaremos el contenido de la columna `pos` y `ds_part` para validar que estas solo tengan dos categorías.

Esto como parte del preprocesamiento a realizar antes de comenzar a trabajar con los textos de la columna `review`.

In [5]:
# Encontrando duplicidad de registros
df.duplicated().sum()

90

Como podemos observar, tenemos `90` registros duplicados los cuales pueden provocar un sesgo en el entrenamiento por lo que es importante excluirlos del `df` a procesar.

In [6]:
# Eliminando registros ducplicados
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [7]:
# Validando columna "pos"
df['pos'].value_counts()

0    23680
1    23561
Name: pos, dtype: int64

In [8]:
# Validando columna "ds_part"
df['ds_part'].value_counts()

train    23758
test     23483
Name: ds_part, dtype: int64

Como podemos observar, tanto la columna `pos` como `ds_part` no cuentan con anomalías en sus datos. Hasta este punto podemos afirmar que tenemos los datos necesarios para poder comenzar a trabajar en el modelo de recomendación basado en las criticas de las películas.

# PLN (Procesamiento de Lenguaje Natural)

Para comenzar a trabajar con las reseñas o críticas, debemos recordar que tanto los modelos matemáticos como los ordenadores, únicamente pueden procesar números, por lo que es importante hacer una transformación de tipo de dato, sin embargo, este proceso va de la mano de ciertos pasos que permitiran hacer dicha conversión eficientemente y son:

- `Tokenización`: Es el proceso mediante el cual las frases se dividen en tokes, es decir, palabras y símbolos separados.
- `Lematización`: Es el proceso mediante el cual se basifican las palabras, es decir, las palabras conjugadas en los diferentes tiempos verbales se transformar en las palabras base.
- `Stopwords`: Son aquellas palabras que no aportan información a las frases, es decir, palabras que no significan nada por si solas y son conocidas como "palabras vacias".

Por cuestiones de procesamiento, haremos uso del proceso `TF-IDF` en el cual a través de la creación de una bolsa de palabras o corpus podemos calcular la frecuencia de aparición de las palabras en un texto y la frecuencia de aparición en el corpus del tal forma que podemos hacer dicha conversión de texto-números de una forma coherente y eficiente para los modelos de machine learning.

Comenzaremos con el proceso de `tokenización` y `lematización` para lo cual tendremos que hacer uso de la librería `NLTK (Natural Language ToolKit)`, ademas también, requeriremos eliminar simbolos ya que no son de utilidad para el algoritmo.

## Tokenización, lematización y eliminación de símbolos

In [9]:
# Proceso de tokenización, lematización y eliminación de símbolos
lemmatizer  = WordNetLemmatizer() 
pattern = r"[^a-zA-Z']"

def tokens_lemmas(text):
    text = re.sub(pattern," ",text)
    tokens = word_tokenize(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    result = ' '.join(lemmas)
    return result

df['lemmas_review'] = df['review'].apply(tokens_lemmas)
df.head(10)

,review,pos,ds_part,lemmas_review
0,The pakage implies that Warren Beatty and Gold...,0,train,the pakage implies that warren beatty and gold...
1,How the hell did they get this made?! Presenti...,0,train,how the hell did they get this made presenting...
2,There is no real story the film seems more lik...,0,test,there is no real story the film seems more lik...
3,Um .... a serious film about troubled teens in...,1,test,um a serious film about troubled teen in singa...
4,I'm totally agree with GarryJohal from Singapo...,1,test,i 'm totally agree with garryjohal from singap...
5,This is the first movie I've seen from Singapo...,1,test,this is the first movie i 've seen from singap...
6,Yes non-Singaporean's can't see what's the big...,1,test,yes non singaporean 's ca n't see what 's the ...
7,This true story of Carlson's Raiders is more o...,0,train,this true story of carlson 's raider is more o...
8,Should have been titled 'Balderdash!' Little i...,0,train,should have been titled 'balderdash ' little i...
9,The movie 'Gung Ho!': The Story of Carlson's M...,0,train,the movie 'gung ho ' the story of carlson 's m...


Ya que tenemos las reseñas de la forma correcta, comenzaremos con la creación de la bolsa de palabras usando `CountVectorizer` de `sklearn` lo cual nos permite crear un corpus de texto en una bolsa de palabras. En este punto al crear la bolsa de palabras debemos tener en consideración que esta debe contemplar únicamente los registros que en la columna `ds_part` tengan el dato `train`.

In [10]:
# Creando bolsa de palabras
stop_words = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
corpus = df[df['ds_part'] == 'train']['lemmas_review']
tf_idf = count_tf_idf.fit(corpus)
X_train = tf_idf.transform(corpus)
X_train.shape

(23758, 64299)

Como podemos observar, la bolsa de palabras ya vectorizada consta de `23,758` registros los cuales coinciden por completo con la cantidad de registros categorizados por la columna `ds_part` como `train`.

Ya que contamos con dicha bolsa de palabras, debemos considerar que para los registros de testeo, estos deben ser modificados bajo las mismas condiciones con las que fue construida la bolsa, es decir, deben ser transformadas a partir de `tf_idf` únicamente.

A partir de este punto ya podemos comenzar con el entrenamiento de modelos de clasificación.

# Modelos de clasificación

Comenzaremos con los modelos de clasificación a partir de este punto empezando por la `regresión logística`.

## Regresión logística

Recordando los parámetros que recibe la `regresión logística`, para el parámetro `solver`, tenemos lo siguiente:

- `liblinear`: Se aplica cuando se tiene un dataset pequeño ademas de una categorización binaria, es decir, uno contra el resto de resultados.
- `newton-cg`, `sag`, `saga` and `lbfgs`: Se aplican cuando se tienen problemas de multiclase, es decir, que la clasificación va más allá de 2 clases de resultados.
- `newton-cholesky`: Se usa cuando dentro del mismo dataset hay información categorica.

Por lo que `liblinear` es el parámetro más adecuado por las características de los registros en `df`.

In [11]:
# Encontrando "y_train"
y_train = df[df['ds_part'] == 'train']['pos']
y_train.shape

(23758,)

In [12]:
# Encontrando X_test, y_test
corpus_test = df[df['ds_part'] == 'test']['lemmas_review']
X_test = tf_idf.transform(corpus_test)
y_test = df[df['ds_part'] == 'test']['pos']
X_test.shape, y_test.shape

((23483, 64299), (23483,))

In [13]:
# Implementando modelo de clasificación
lr_model = LogisticRegression(random_state=12345,solver='liblinear')
lr_model.fit(X_train,y_train)
y_pred_lr = lr_model.predict(X_test)
lr_score = f1_score(y_test,y_pred_lr)
print('La precisión obtenida en el algoritmo de "Regresión logística" es:',lr_score)

La precisión obtenida en el algoritmo de "Regresión logística es": 0.8792402789066176


Como podemos observar, obtenemos un resultado satisfactorio pues cumple con la condición establecida lo que nos quiere decir que la técnica de `procesamiento de lenguaje natural` es suficientemente buena para asegurar una exactitud de predicción aceptable. Aunque evaluaremos otros algoritmos de clasificación para que de igual forma obtengamos su desempeño.

## Árboles de decisión

In [22]:
# Ajustando parámetros
dtc_model = DecisionTreeClassifier(random_state=12345)
params = {'criterion': ['gini','entropy'],
          'splitter': ['best'],
          'max_depth': range(1,10), 
          'min_samples_split': range(2,20,2), 
          'min_samples_leaf': range(2,20,2)}
grid = GridSearchCV(dtc_model,params,cv=2,n_jobs=-1,scoring='f1')
grid.fit(X_train,y_train)
print("Best parameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
print("Best estimator: ", grid.best_estimator_)

Best parameters:  {'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 16, 'min_samples_split': 2, 'splitter': 'best'}
Best score:  0.7562507618291743
Best estimator:  DecisionTreeClassifier(max_depth=9, min_samples_leaf=16, random_state=12345)


In [23]:
# Implementando modelo de clasificación
dtc_model = DecisionTreeClassifier(random_state=12345, criterion='gini', max_depth=9,
                                   min_samples_leaf=16, min_samples_split=2,splitter='best')
dtc_model.fit(X_train,y_train)
y_pred_dtc = dtc_model.predict(X_test)
dtc_score = f1_score(y_test,y_pred_dtc)
print('La precisión obtenida en el algoritmo de "Árboles de decisión" es:',dtc_score)

La precisión obtenida en el algoritmo de "Regresión logística" es: 0.7565024479804162


## Bosques aleatorios

In [25]:
# Ajustando parámetros
rfc_model = RandomForestClassifier(random_state=12345)
params = {'criterion': ['gini','entropy'],
          'max_depth': range(1,10),
          'min_samples_split': range(2,20,2), 
          'min_samples_leaf': range(2,20,2)}
grid = GridSearchCV(rfc_model,params,cv=2,n_jobs=-1,scoring='f1')
grid.fit(X_train,y_train)
print("Best parameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
print("Best estimator: ", grid.best_estimator_)

Best parameters:  {'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 6, 'min_samples_split': 16, 'n_estimators': 151}
Best score:  0.8309877083727699
Best estimator:  RandomForestClassifier(max_depth=9, min_samples_leaf=6, min_samples_split=16,
                       n_estimators=151, random_state=12345)


In [26]:
# Implementando modelo de clasificación
rfc_model = RandomForestClassifier(random_state=12345,criterion='gini',max_depth=9,
                                   min_samples_leaf=6,min_samples_split=16,n_jobs=-1)
rfc_model.fit(X_train,y_train)
y_pred_rfc = rfc_model.predict(X_test)
rfc_score = f1_score(y_test,y_pred_rfc)
print('La precisión obtenida en el algoritmo de "Bosques aleatorios" es:',rfc_score)

La precisión obtenida en el algoritmo de "Bosques aleatorios" es: 0.8243831640058056


Como podemos observar en tanto en `árboles de decisión` como en `bosques aleatorios`, se tiene un desempeño que se encuentra por debajo de la condicionante para considerarlo un modelo aceptablel, esto puede deberse a las limitantes de procesamiento por parte del ordenador ya que a mayor cantidad de parámetros se vuelve un proceso más y más complejo ademas de demandar consumo de procesamiento cada vez mayor.

Sin embargo, aun tenemos un modelo más por probar que es un algoritmo que potencializa el gradiente.

## LightGBM

In [32]:
# Modelo de clasificación con potenciación de gradiente
train_data = lgb.Dataset(X_train,label=y_train)
test_data = lgb.Dataset(X_test,label=y_test)

gbm_model = lgb.LGBMClassifier()

params = {'learning_rate': [0.05, 0.1, 0.2],
          'num_leaves': [20, 31, 40],
          'n_estimators': [50, 100, 200]}

grid = GridSearchCV(gbm_model,params,cv=2,n_jobs=-1,scoring='f1')
grid.fit(X_train,y_train)
print("Best parameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
print("Best estimator: ", grid.best_estimator_)

Best parameters:  {'learning_rate': 0.1, 'n_estimators': 200, 'num_leaves': 40}
Best score:  0.8579249339910451
Best estimator:  LGBMClassifier(n_estimators=200, num_leaves=40)


In [33]:
# Implementando modelo de clasificación
gbm_model = lgb.LGBMClassifier(learning_rate=0.1, num_leaves=40, n_estimators=200)
gbm_model.fit(X_train,y_train)
y_pred_gbm = gbm_model.predict(X_test)
gbm_score = f1_score(y_test,y_pred_gbm)
print('La precisión obtenida en el algoritmo de "LightGBM" es:',gbm_score)

La precisión obtenida en el algoritmo de "LightGBM" es: 0.8733208637986736


Como podemos observar, tanto `regresión logística` como `lightGBM` son los modelos que nos permiten una eficiencia de clasificación aceptable para la `Film Junky Union`. 

In [41]:
# Tabla de desempeño
pd.DataFrame(data={'Desempeño': [lr_score*100,dtc_score*100,rfc_score*100,gbm_score*100]},
             index=['Regresión logística','Árboles de decisión','Bosques aleatorios','LightGBM']).sort_values(by=['Desempeño'],ascending=False)

,Desempeño
Regresión logística,87.924028
LightGBM,87.332086
Bosques aleatorios,82.438316
Árboles de decisión,75.650245


# Conclusiones

Al trabajar este tipo de proyectos, es importante considerar la capacidad computacional del ordenador en el que uno se encuentre trabajando, esto es debido a que se trabaja con matrices lo cuál demanda capacidad de computo, por lo que se opto por trabajar con la creación de bolsa de palabras a partir de un corpus bajo el cálculo de `TF-IDF`, por otro lado tenemos los modelos de clasificación que pudimos observar que aquellos modelos basados en árboles, presentan una eficiencia menor contra la `regresión logística` y `LightGBM`, esto puede deberse a lo anteriormente menciona pues la capacidades de `árboles de decisión` y `bosques aleatorios` se ven limitadas por le host de procesamiento que en este caso es el ordenador personal.